#  Study basins of attraction using reversed models


In this notebook we combine [bioLQM](http://colomoto.org/biolqm) and [BoolSim](https://www.vital-it.ch/research/software/boolSim) to study the basins of attraction of qualitative models.

In [1]:
import boolsim
import biolqm

Note: the boolsim.py file provides a simple Python API for a subset of BoolSim command-line features.
We plan to further extend this API and to integrate it directly in the CoLoMoTo Docker image.

## Load and reverse the model

Load the model, compute the reversed version and export both to boolsim

In [2]:
model_name = "calzone2010/Calzone_2010_reduced"

model = biolqm.load(f"{model_name}.sbml")
reverse = biolqm.reverse(model)
boolsim_model = boolsim.BoolSimModel(biolqm.biolqm.save(model, f"{model_name}.boolsim"))
boolsim_model_rev = boolsim.BoolSimModel(biolqm.biolqm.save(reverse, f"{model_name}_reversed.boolsim"))

## Identify all attractors

In [3]:
%%time

attractors = boolsim_model.attractors()

for a in attractors:
    a.show()

apoptosome ATP BAX BCL- CASP3 CASP8 cFLIP cIAP Cyt_c IKK MOMP MPT NFkB RIP1 RIP1K RIP1ub ROS SMAC XIAP
0 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 0 1

apoptosome ATP BAX BCL- CASP3 CASP8 cFLIP cIAP Cyt_c IKK MOMP MPT NFkB RIP1 RIP1K RIP1ub ROS SMAC XIAP
0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 0

apoptosome ATP BAX BCL- CASP3 CASP8 cFLIP cIAP Cyt_c IKK MOMP MPT NFkB RIP1 RIP1K RIP1ub ROS SMAC XIAP
1 1 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0

CPU times: user 2.54 ms, sys: 5.98 ms, total: 8.51 ms
Wall time: 235 ms


## Basins of attractions

Find the weak and strong basins of attraction for each identified attractor.
* Weak basins: all states which can lead to this attractor
* Strong basins: the subset of the weak basin which can not reach any other attractor

In [4]:
%%time
weaks = [ boolsim_model_rev.reachable(a, suffix="weak") for a in attractors  ]

CPU times: user 3.48 ms, sys: 7.55 ms, total: 11 ms
Wall time: 511 ms


In [5]:
%%time
strongs = []
for w in weaks:
    excluded = [ e for e in weaks if e != w ]
    output = w.statefile.replace("weak","strong")
    strongs.append( w.exclude(output, excluded) )

CPU times: user 1.3 ms, sys: 11.7 ms, total: 13 ms
Wall time: 54.2 ms


## Boundaries

The boundary of a basin is the set of transition from an external state to an internal one (by definition, there is no transition from internal state to external ones). Here we identify the set of external and internal states of these transitions.

The external boundary is the set of states outside of the basin which can reach the basin in a single step. We identify them by computing the set of states reachable in one reverse step from the basin, excluding states of the basin.

The internal boundary is the set of states of the basin reachable in one step from the external boundary.

In [6]:
%%time

externals = []
internals = []
for s in strongs:
    output = s.statefile.replace("strong","externalboundary")
    tmp = boolsim_model_rev.reachable(s, max_steps=1)
    ext = tmp.exclude(output, [s,] )
    
    output = s.statefile.replace("strong","internalboundary")
    tmp = boolsim_model.reachable(ext, max_steps=1)
    internal = tmp.intersect( output, s )
    
    externals.append( ext )
    internals.append( internal )

CPU times: user 6.59 ms, sys: 26.4 ms, total: 33 ms
Wall time: 613 ms


## Simplify sets of states with espresso

BoolSim manipulates sets of states as ordered decision diagrams, which can be listed as a collection of patterns. However, these lists of patterns may not provide the most compact representation of the set of states. To simplify these lists, we can use an heuristic method implemented in the classical [espresso](https://en.wikipedia.org/wiki/Espresso_heuristic_logic_minimizer) tool.

In [7]:
strongs[1].show()
strongs[1].simplify().show()

apoptosome ATP BAX BCL- CASP3 CASP8 cFLIP cIAP Cyt_c IKK MOMP MPT NFkB RIP1 RIP1K RIP1ub ROS SMAC XIAP
- - - 0 - 0 - 0 - 0 - 1 0 - - 0 1 - -
- - - 0 - 1 0 0 - 0 - 1 0 - - 0 1 - -
- - - 0 - 1 0 1 - 0 - 1 0 0 - 0 1 - -
- - - 0 - 1 1 0 - 0 - 1 0 - - 0 1 - -

apoptosome ATP BAX BCL- CASP3 CASP8 cFLIP cIAP Cyt_c IKK MOMP MPT NFkB RIP1 RIP1K RIP1ub ROS SMAC XIAP
- - - 0 - 1 0 - - 0 - 1 0 0 - 0 1 - -
- - - 0 - - - 0 - 0 - 1 0 - - 0 1 - -



# Some statistics

Show the size of all basins and their boundaries

In [8]:
for attractor, weak,strong, external, internal in zip(attractors, weaks,strongs, externals, internals):
    print(f"attractor: {attractor.count()}")
    print(f"strong: {strong.count()}")
    print(f"int: {internal.count()}")
    print(f"ext: {external.count()}")
    print()

attractor: 1
strong: 60
int: 60
ext: 776

attractor: 1
strong: 4608
int: 4608
ext: 30208

attractor: 1
strong: 1036
int: 1036
ext: 9324

